In [1]:
from itertools import product
import numpy as np
from pulp import LpMinimize, LpMaximize, LpProblem
from funcs import make_partition_probabilities, simulate_participants, get_state_probabilities, \
                  set_up_variables, solve, itt_ate, q_ate, true_ate

In [2]:
compliance_partitions = ['always_taker', 'complier', 'defier', 'never_taker']
response_partitions = ['always_better', 'helped', 'hurt', 'never_better']
partition_types = np.array(list(product(compliance_partitions, response_partitions)))

In [3]:
partition_types

array([['always_taker', 'always_better'],
       ['always_taker', 'helped'],
       ['always_taker', 'hurt'],
       ['always_taker', 'never_better'],
       ['complier', 'always_better'],
       ['complier', 'helped'],
       ['complier', 'hurt'],
       ['complier', 'never_better'],
       ['defier', 'always_better'],
       ['defier', 'helped'],
       ['defier', 'hurt'],
       ['defier', 'never_better'],
       ['never_taker', 'always_better'],
       ['never_taker', 'helped'],
       ['never_taker', 'hurt'],
       ['never_taker', 'never_better']], dtype='<U13')

In [22]:
def experiment(distribution_floats, n_participants, partition_types):
    partition_p = make_partition_probabilities(distribution_floats)
    df = simulate_participants(partition_p, n_participants, partition_types)
    
    itt_effect = itt_ate(df)
    
    state_p = get_state_probabilities(df)
    q = set_up_variables(partition_types)
    minimize = LpProblem('min', LpMinimize)
    maximize = LpProblem('max', LpMaximize)
    
    q_min = solve(minimize, q, state_p)
    q_max = solve(maximize, q, state_p)
    
    lower_ate = q_ate(q_min)
    upper_ate = q_ate(q_max)
    
    true = true_ate(df)
    
    return true, itt_effect, lower_ate, upper_ate

In [40]:
distribution_magnitude = [0, 10, 0, 0, 0, 10, 0, 0, 0, 10, 0, 0, 0, 10, 0, 0]
n_participants = 20000

In [41]:
experiment(distribution_magnitude, n_participants, partition_types)

(1.0, -0.0012999999999999678, 0.00065, 1.0)

In [44]:
distribution_magnitude = [0, 0, 0, 0, 0, 10, 4, 0, 0, 10, 4, 0, 0, 10, 4, 0]
experiment(distribution_magnitude, n_participants, partition_types)

(0.4234, -0.005900000000000016, -0.04305, 0.9522499999999999)

In [ ]:
distribution_magnitude = [0, 0, 0, 0, 0, 10, 4, 0, 0, 10, 4, 0, 0, 10, 4, 0]
experiment(distribution_magnitude, n_participants, partition_types)